# <font color = 'gold'> Steps to go on in this part of the automation: </font>

<font color = 'teal'>

- Filter the itens that were used in the last 24 months
- Filter all the itens that the status is "ativo"
- Make sure that duplicated itens is no more

</font>

In [1]:
import pandas as pd
import re

def treating_columns(df):
    columns = df.columns
    columns = columns.str.lower()
    df.columns = columns
    return df


def text_treating(df, columns = []):
    for c in columns:
        df[c] = df[c].str.lower()
    return df


def filter_itens(df, column_movimentation, column_status):
    # applying the first and second step in the header
    assert (df[column_movimentation].nunique() == 2) and (df[column_status].nunique() == 2)

    df[column_movimentation] = df[column_movimentation].str.upper()
    df[column_status] = df[column_status].str.upper()

    f = (df[column_movimentation] == 'SIM') & (df[column_status] == 'ATIVO')

    return df.loc[f]

def removing_duplicates(df, column_to_clean):
    values = df.groupby(column_to_clean).cumcount() + 1
    
    df['row_number'] = values

    return df.loc[df.row_number == 1]


def regex_reference(text):
    pattern =  r'\[([^]]+)\]'
    text_treated = re.findall(pattern = pattern, string = text)
    
    if len(text_treated) != 0:
        return re.sub(r'[-.,/|;]', '', text_treated[0])
    else:
        return None


In [2]:
clean_system_legacy = pd.read_excel(io = 'files/itens_to_clean.xlsx', sheet_name=0)

clean_system_legacy_v2 = treating_columns(clean_system_legacy)
clean_system_legacy_v2 = filter_itens(clean_system_legacy_v2, column_movimentation = 'movimentacao_24', column_status = 'situacao_produto')
clean_system_legacy_v2 = removing_duplicates(clean_system_legacy_v2, column_to_clean = 'desc_cadastro')

# APLICACAO EM AMBOS OS SISTEMAS
#clean_system_legacy_v2 = treating_columns(clean_system_legacy)
columns_to_treat = ['desc_cadastro', 'desc_grupo']
clean_system_legacy_v2 = text_treating(clean_system_legacy_v2, columns = columns_to_treat)
clean_system_legacy_v2['referencia'] = clean_system_legacy_v2.desc_cadastro.apply(regex_reference)

/var/folders/mq/g6k1b1h96tsb47xpdljnctsc0000gp/T/ipykernel_47957/1682609309.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['row_number'] = values


In [3]:
clean_system_legacy_v2.head()

,cod_pro,desc_cadastro,movimentacao_12,movimentacao_24,referencia,saldo,codigo_grupo,desc_grupo,custo_medio,custo_atual,...,brasindice,cod_bras_conc,tuss,fornecedor,fabricante_medicamento,padronizado,consignado,fabricante,situacao_produto,row_number
0,2,agulha descartavel 40x12,SIM,SIM,None,1824.0,2,material hospitalar,0.12,0.12,...,NaN,NaN,78420130,000198-MACROMED C.M. MED .HOSP. LTDA,NaN,S,N,SOL-MILLEN,ATIVO,1
1,3,cateter eletrodo estim bip temporario 5fr 10x1...,SIM,SIM,24tpls151,2.0,19,material especial,330.46,356.60,...,NaN,NaN,78231060,000330-BIOMEDICAL PRODUTOS CIENTIFICOS MEDICOS,NEUROMEDEX | ALEMANHA,S,N,BIO MINAS,ATIVO,1
3,6,carnes | file de frango,NÃO,SIM,None,0.0,25,frigorificado,18.58,19.80,...,NaN,NaN,NaN,001285-EMPORIO MEGA 100 COM. DE ALIMENTOS S.A.,NaN,S,N,NaN,ATIVO,1
9,12,dieta enteral nutrison advanced diason energy ...,SIM,SIM,None,0.0,8,preparos p/ nutricao,40.40,2075.00,...,NaN,NaN,NaN,000272-NUTRIPORT COMERCIAL LTDA,NaN,S,N,NaN,ATIVO,1
10,13,dieta enteral nutrison energy multi fiber 1.5 ...,SIM,SIM,None,0.0,8,preparos p/ nutricao,37.27,1235.00,...,NaN,NaN,NaN,000272-NUTRIPORT COMERCIAL LTDA,NaN,S,N,NaN,ATIVO,1


# Second Step


- a gente pega a desc do produto e o cód e coloca na outra planilha
- 

In [4]:
df = pd.read_excel(io = 'files/final_step.xlsx', sheet_name=1)

In [5]:
df = treating_columns(df)
columns_to_treat = ['descricao', 'descricao_completa']
df = text_treating(df, columns = columns_to_treat)
df['referencia_tratada'] = df.descricao_completa.apply(regex_reference) # fazer para descricao também

df.head()

,cod_p12,descricao,descricao_completa,descricao_clinica,referencia,cod_tipo,cod_fabricante,simpro,tuss,brasindice,anvisa,dt_validade_anvisa,um_compra,um_estoque,um_consumo,dt_inclusao,dt_atualizacao,status,motivo_bloqueio,referencia_tratada
0,10004,cocacola lata cx c/12 unid,cocacola lata cx c/12 unid ...,...,...,77,002586,,,,,,CX,CX,CX,20180112 02:45:58,20220519 10:02:44,INATIVO,NaN,None
1,10005,cocada branca cremosa bd c/ 10 kg,cocada branca cremosa bd c/ 10 kg ...,...,...,77,002586,,,,,,UN,UN,UN,20180112 02:45:58,20221021 21:07:30,ATIVO,NaN,None
2,1,macronebulizador policarbonato,macronebulizador policarbonato ...,...,000108 ...,63,002586,,,,.,,UN,UN,UN,20180112 02:45:58,20230909 11:57:03,INATIVO,?,None
3,10,frasco dieta nutrimed 100ml,frasco dieta enteral nutrimed 100ml ...,...,0007-8 ...,10,000433,0000286341,,,10442360012,,UN,UN,UN,20180112 02:45:58,20210304 23:44:04,ATIVO,NaN,None
4,100,castelo maa jir10d rosca 1/2,castelo maa jir10d rosca de 1/2 ...,...,01468 ...,53,001186,,,,,,UN,UN,UN,20180112 02:45:58,20221021 21:04:15,ATIVO,NaN,None


In [6]:
l_key = 'referencia'
r_key = 'referencia_tratada'

cols = ['cod_p12', 'descricao_completa', 'referencia_tratada']
df_left = df[cols]


cols = ['desc_cadastro', 'referencia']
df_right = clean_system_legacy_v2[cols]


df_final = df_left.merge(right = df_right, how = 'left', left_on = l_key, right_on = r_key)

KeyError: 'referencia_tratada'